In [1]:
import logging
import os
import sys
import json
from typing import List

# Note: The below two lines are for local testing only and will not be added to the PR
_src_path = os.path.dirname('/Volumes/workplace/LEX/bedRock/open_source/langchain-aws/langchain-aws/libs/aws/langchain_aws')
sys.path.insert(0, _src_path)

from langchain.agents.agent import AgentExecutor
from langchain_core.tools import Tool, StructuredTool
from langchain_aws.agents.bedrock.agent_base import agent_tool, BedrockAgentMetadata, BedrockAgentBase
from langchain_aws.agents.bedrock.agent_executor import BedrockAgentExecutor
from langchain_aws.agents.bedrock.bedrock_agent import BedrockAgent

logging.basicConfig(format='%(message)s', level=logging.INFO)

In [2]:

agent_name: str = "MortgageEvaluatorAgent_1"
agent_instruction: str = "You are an agent who helps with getting the mortgage rate based on the current asset valuation"
agent_region: str = "us-east-1"

@agent_tool(action_group='AssetDetail', action='getAssetValue')
def getAssetValue(
        asset_holder_id: str = ' '
) -> str:
    """
    Get the asset value for an owner id

    Args:
        asset_holder_id: id of the owner holding the asset
    Returns:
        str -> the valuation of the asset
    """
    return f"The total asset value for {asset_holder_id} is 100K"


@agent_tool
def getMortgageRate(
        asset_holder_id: str = ' ',
        asset_value: str = ' '
) -> str:
    """
    Get the mortage rate based on asset value

    Args:
        asset_holder_id: id of the owner holding the asset
        asset_value: asset value which is used to get the mortgage rate
    Returns:
        str -> the calculated mortage rate based on the asset value
    """
    return f"The mortgage rate for {asset_holder_id} with asset value of {asset_value} is 8.87%"

agent_tools: List[Tool] = [getAssetValue, getMortgageRate]

def handle_trace(event=dict):
    logging.info("\n********************** Event **********************\n")
    logging.info(f"{json.dumps(event, indent=4)} \n")

In [3]:
bedrock_agent = BedrockAgent(
    agent_name=agent_name,
    agent_instruction=agent_instruction,
    agent_region=agent_region,
    agent_tools=agent_tools,
    trace_handler=handle_trace
)

agent_executor = BedrockAgentExecutor.from_agent_and_tools(
    agent=bedrock_agent,
    verbose=True,
    tools=agent_tools,
    return_intermediate_steps=True,
    max_iterations=8
)

Found credentials in shared credentials file: ~/.aws/credentials
Created AWS Bedrock agent with request id: 9aea4deb-5dbf-48bc-9e4f-358b858987b6
Created Bedrock agent ActionGroup AssetDetail and Function with request id: 7f200a8b-56af-4b9e-871d-71eb1352fcfd
Created Bedrock agent ActionGroup getMortgageRateActionGroup and Function with request id: 3b533093-a4a8-4ef2-b970-c9b6e03ad819
Prepared AWS Bedrock agent with request id: 052ef1b7-08ff-4c01-aa6f-ea1b9f583307


In [4]:
agent_executor.invoke({"input": "what is my mortgage rate for id AVC-1234"})



> Entering new BedrockAgentExecutor chain...


Bedrock agent call request id: cd590b70-50d1-495f-893c-d754c8964130

********************** Bedrock Agents Input **********************

Input: what is my mortgage rate for id AVC-1234 

SessionState: {} 


********************** Event **********************

{
    "rationale": {
        "text": "To get the mortgage rate for the asset holder with id AVC-1234, I first need to get the asset value for that id. Then I can use that asset value to determine the mortgage rate.",
        "traceId": "cd590b70-50d1-495f-893c-d754c8964130-0"
    }
} 


********************** Event **********************

{
    "invocationInput": {
        "actionGroupInvocationInput": {
            "actionGroupName": "AssetDetail",
            "function": "getAssetValue",
            "parameters": [
                {
                    "name": "asset_holder_id",
                    "type": "string",
                    "value": "AVC-1234"
                }
            ]
        },
        "invocationType": "ACTI


> Finished chain.


{'input': 'what is my mortgage rate for id AVC-1234',
 'output': 'The mortgage rate for the asset holder with id AVC-1234 and an asset value of 100K is 8.87%.'}

In [5]:
bedrock_agent.delete()

Deleted Bedrock agent call request id: e6d5a5ff-e8df-4cb0-890f-86c2640f7834
